# Chapter 11

In the previous chapter, we introduced different instructions that manipulate and transform different stores of data like `sets`, `maps` and `big maps`. We learned how to create these stores, how to save new values inside and how to get them out. However, we left aside one of the most powerful features of this type of data. Instead of manipulating the values one by one, what if we'd loop through the whole structure and gain access to all the data at once? 🤯

This chapter is about instructions that will help us manipulate all the values in sets, maps or big maps, whether we want to read those values, use them or modify them. As you will notice throughout the chapter, the instructions we use are sometimes the same and apply to different types, although they may be used differently. This is why you have to remember precisely how the instruction affects the structure on the stack and how the elements must be ordered on the stack before calling the instruction you want.

## The **`LOOP`** instruction

Most of widely used programming languages have loops so you may be familiar with the concept. In a nutshell, loops consist of a piece of code that repeats itself until a certain condition is met. This can also be done in Michelson. Here is a very simple example of a loop:

In [12]:
DEBUG False ;
storage unit ;
parameter unit ;
BEGIN Unit Unit ;
DROP ;
PUSH int 0 ;
PUSH bool True ;
LOOP {  
    PUSH int 1 ; 
    ADD ; 
    DUP ;
    PUSH int 5 ;
    CMPNEQ ;
    } ;
DUMP ;

value,type
5,int


This is a naive example to demonstrate how loops work. One of the main characteristics of loops is that they "sit" on a boolean value. As long as the boolean value is `True`, the loop will keep executing the code inside. At the end of the loop, you must have at least a boolean value to tell the loop if it must continue iterating or not. 

In this example, we push `int 0` to the stack and `True` before starting the loop. Then, we push `int 1`, add it to the value present on the stack, duplicate it to keep the value on the stack, push `int 5` and compare the 2 values. **`CMPNEQ`** checks if the value we got from the addition is not equal to `5`. Before we reach `5`, the instruction returns `True`, which keeps the loop iterating. As soon as the value is `5`, **`CMPNEQ`** returns `False` and the loop stops.

Loops can become more interesting if you use them with values like lists where you will be able to fetch every element of the list and use them however you want. Let's crank the difficulty up a notch and imagine we want to concatenate all the strings in a list:

In [15]:
DEBUG False;
storage unit ;
parameter unit ;
BEGIN Unit Unit ;
DROP ;
PUSH string "" ;
PUSH (list string) {"hello " ; "world " ; "and " ; "Tezos"} ;
PUSH bool True ;
LOOP { 
    IF_CONS
        { SWAP ; DIP { SWAP ; CONCAT } ; PUSH bool True }
        { PUSH bool False } ;
    } ; 
DUMP ;

value,type
"""hello world and Tezos""",string


This one is a little more complex as it makes it more difficult to follow the different changes of state of the stack. 

Before the loop, the stack is made of three blocks: a `bool True` block necessary to start the loop, a `(list string` containing the strings we want to concatenate together and an empty `string` we will use for the first concatenation.

On the first iteration, the `bool True` block is removed from the stack, which exposes the list below. **`IF_CONS`** pops the head of the list, pushes the tail onto the stack and pushes the head above if the list is not empty. At this point, the stack is made of a `string`, a `(list string)` and the empty `string`. **`SWAP`** puts the two strings next to each other and **`DIP { SWAP ; CONCAT }`** concatenates them. We then push `bool True` to continue the execution of the loop which will end only when the list is empty.